In [1]:
import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import backend as K

In [2]:
IMG_SIZE = 512
BATCH_SIZE = 1
N_CLASSES = 2  # Change this for multi-class
EPOCHS = 30
LEARNING_RATE = 0.0001
print("done")

done


In [3]:
DATASET_PATH = "Levir-CD/"
TRAIN_IMG1_DIR = os.path.join(DATASET_PATH, 'train/A/A/')
TRAIN_IMG2_DIR = os.path.join(DATASET_PATH, 'train/B/B/')
TRAIN_MASK_DIR = os.path.join(DATASET_PATH, 'train/L/label/')
VAL_IMG1_DIR = os.path.join(DATASET_PATH, 'val/A/A/')
VAL_IMG2_DIR = os.path.join(DATASET_PATH, 'val/B/B/')
VAL_MASK_DIR = os.path.join(DATASET_PATH, 'val/L/label/')
print("done")

done


In [4]:
import tensorflow as tf
import os

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
tf.keras.backend.clear_session()
tf.compat.v1.reset_default_graph()

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices([], 'GPU')
        
        tf.config.set_visible_devices(gpus, 'GPU')
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        
        print(f"Configured {len(gpus)} GPU(s) with memory growth enabled")
    except RuntimeError as e:
        print("GPU configuration failed:", e)

try:
    tf.compat.v1.disable_eager_execution()
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))
except Exception as e:
    print("TF1 compatibility config failed:", e)

Configured 1 GPU(s) with memory growth enabled



In [5]:
def load_images(image_dir, mask=False):
    images = []
    for img_name in sorted(os.listdir(image_dir)):
        if mask:
            # Load mask as grayscale
            img = cv2.imread(os.path.join(image_dir, img_name), cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            
            if N_CLASSES == 2:  # Binary case
                img = (img > 127).astype(np.float32)  # Single channel [0,1]
                img = np.expand_dims(img, axis=-1)     # Shape (H,W,1)
            else:  # Multi-class case
                img = (img / 255 * (N_CLASSES-1)).astype(np.uint8)  # Scale to class indices
                img = tf.keras.utils.to_categorical(img, num_classes=N_CLASSES)  # One-hot
        else:
            # Load regular image
            img = cv2.imread(os.path.join(image_dir, img_name))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
            img = img.astype(np.float32) / 255.0
        images.append(img)
    return np.array(images)

X1_train = load_images(TRAIN_IMG1_DIR) / 255.0
X2_train = load_images(TRAIN_IMG2_DIR) / 255.0
y_train = load_images(TRAIN_MASK_DIR, mask=True)

X1_val = load_images(VAL_IMG1_DIR) / 255.0
X2_val = load_images(VAL_IMG2_DIR) / 255.0
y_val = load_images(VAL_MASK_DIR, mask=True)
print("Training data shapes:")
print(f"X1: {X1_train.shape}, X2: {X2_train.shape}, y: {y_train.shape}")
print("\nValidation data shapes:")
print(f"X1: {X1_val.shape}, X2: {X2_val.shape}, y: {y_val.shape}")

Training data shapes:
X1: (445, 512, 512, 3), X2: (445, 512, 512, 3), y: (445, 512, 512, 1)

Validation data shapes:
X1: (64, 512, 512, 3), X2: (64, 512, 512, 3), y: (64, 512, 512, 1)


In [6]:
def FocalLoss(targets, inputs, alpha=0.8, gamma=2):    
    inputs = K.flatten(inputs)
    targets = K.flatten(targets)
    BCE = K.binary_crossentropy(targets, inputs)
    BCE_EXP = K.exp(-BCE)
    focal_loss = K.mean(alpha * K.pow((1-BCE_EXP), gamma) * BCE)
    return focal_loss

def att_module(x):
    x1 = AveragePooling2D((x.shape[1], x.shape[1]), strides=None, padding="same")(x)
    x1 = Conv1D(x.shape[-1], 3, padding='same', kernel_initializer='random_normal')(x1)
    x1 = Activation('sigmoid')(x1)
    x1 = Multiply()([x1, x])
    x2 = K.max(x, axis=-1)
    x2 = tf.expand_dims(x2, axis=-1)
    x2 = Conv2D(x.shape[-1], 3, padding='same', kernel_initializer='random_normal')(x2)
    x2 = Activation('sigmoid')(x2)
    x2 = Multiply()([x2, x])
    x = Add()([x1, x2])
    return x

def conv_block(x, f):
    c = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='random_normal')(x)
    c = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='random_normal')(c)
    c = Conv2D(f, 3, activation='relu', padding='same', kernel_initializer='random_normal')(c)
    return c

def conc_att_conv(x1, x2, x3, f):
    up = Conv2D(f, 2, activation='relu', padding='same', kernel_initializer='random_normal')(UpSampling2D(size=(2,2))(x2))
    merge = concatenate([x1, up, x3], axis=3)
    att = att_module(merge)
    x = conv_block(att, f)
    return x

def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def adsnet(shape=(IMG_SIZE, IMG_SIZE, 3), deep_supervision=True):
    # Input layers
    input_layer_1 = Input(shape)
    input_layer_2 = Input(shape)
    
    # Encoder Pathway
    conv1 = Conv2D(8, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c11 = conv1(input_layer_1)
    c21 = conv1(input_layer_2)
    pool11 = MaxPooling2D(pool_size=(2, 2))(c11)
    pool21 = MaxPooling2D(pool_size=(2, 2))(c21)
    
    conv2 = Conv2D(16, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c12 = conv2(pool11)
    c22 = conv2(pool21)
    pool12 = MaxPooling2D(pool_size=(2, 2))(c12)
    pool22 = MaxPooling2D(pool_size=(2, 2))(c22)   
    
    conv3 = Conv2D(32, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c13 = conv3(pool12)
    c23 = conv3(pool22)
    conv3 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c13 = conv3(c13)
    c23 = conv3(c23)
    pool13 = MaxPooling2D(pool_size=(2, 2))(c13)
    pool23 = MaxPooling2D(pool_size=(2, 2))(c23)
    
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c14 = conv4(pool13)
    c24 = conv4(pool23)
    conv4 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c14 = conv4(c14)
    c24 = conv4(c24)
    pool14 = MaxPooling2D(pool_size=(2, 2))(c14)
    pool24 = MaxPooling2D(pool_size=(2, 2))(c24)
    
    conv5 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c15 = conv5(pool14)
    c25 = conv5(pool24)  
    conv5 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='random_normal')
    c15 = conv5(c15)
    c25 = conv5(c25)
    
    # Difference Maps
    c5 = c15 - c25
    c4 = c14 - c24
    c3 = c13 - c23
    c2 = c12 - c22
    
    # Decoder Pathway with Attention
    merge5 = concatenate([c15, c5, c25], axis=3)
    att51 = att_module(merge5)
    c51 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='random_normal')(att51)
    c51 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='random_normal')(c51)
    
    c52 = conc_att_conv(c14, c51, c24, 64)
    c53 = conc_att_conv(c13, c52, c23, 32)
    c54 = conc_att_conv(c12, c53, c22, 16)
    x3 = conc_att_conv(c11, c54, c21, 8)
    
    merge6 = concatenate([c14, c4, c24], axis=3)
    att61 = att_module(merge6)
    c61 = conv_block(att61, 64)
    c62 = conc_att_conv(c13, c61, c23, 32)
    c63 = conc_att_conv(c12, c62, c22, 16)
    x2 = conc_att_conv(c11, c63, c21, 8)
    
    merge7 = concatenate([c13, c3, c23], axis=3)
    att71 = att_module(merge7)
    c71 = conv_block(att71, 32)
    c72 = conc_att_conv(c12, c71, c22, 16)
    x4 = conc_att_conv(c11, c72, c21, 8)
    
    merge8 = concatenate([c12, c2, c22], axis=3)
    att81 = att_module(merge8)
    c81 = conv_block(att81, 16)
    x1 = conc_att_conv(c11, c81, c21, 8)
    
    # Output Layers
    x1 = Conv2D(N_CLASSES, (1,1), activation='softmax', padding='same')(x1)
    x2 = Conv2D(N_CLASSES, (1,1), activation='softmax', padding='same')(x2)
    x3 = Conv2D(N_CLASSES, (1,1), activation='softmax', padding='same')(x3)    
    x4 = Conv2D(N_CLASSES, (1,1), activation='softmax', padding='same')(x4)
    
    if deep_supervision:
        model = Model([input_layer_1, input_layer_2], [x1, x2, x3, x4])
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                    loss=[FocalLoss]*4,
                    metrics=[get_f1, 'accuracy'])
    else:
        model = Model([input_layer_1, input_layer_2], [x3])
        model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), 
                    loss=FocalLoss,
                    metrics=['accuracy'])

    return model

model = adsnet()
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 8)  224         input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 256, 256, 8)  0           conv2d[0][0]                 

In [8]:
from tensorflow.keras.callbacks import Callback

y_train = y_train.astype(np.float32)
y_val = y_val.astype(np.float32)


class F1Metrics(Callback):
    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        if len(model.outputs) > 1:
            print('\nF1 Scores:')
            for i in range(len(model.outputs)):
                print(f'Output {i+1}: {logs.get(f"get_f1_{i}", 0):.4f}', end=' | ')
        else:
            print(f'\nF1: {logs.get("get_f1", 0):.4f}')
        print(f'Loss: {logs.get("loss", 0):.4f}')
f1_metrics = F1Metrics()

# Callbacks
checkpoint = ModelCheckpoint(
    os.path.join(DATASET_PATH, '00_dsnet3.h5'),
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=True
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

history = model.fit(
    [X1_train, X2_train],
    [y_train, y_train, y_train, y_train],  
    validation_data=(
        [X1_val, X2_val],
        [y_val, y_val, y_val, y_val]
    ),
    epochs=50,
    batch_size=1,
    callbacks=[checkpoint, early_stop, f1_metrics]
)

Train on 445 samples, validate on 64 samples
Epoch 1/50


InvalidArgumentError: 2 root error(s) found.
  (0) Invalid argument: required broadcastable shapes at loc(unknown)
	 [[{{node loss/conv2d_73_loss/mul}}]]
	 [[loss/Identity_2/_1403]]
  (1) Invalid argument: required broadcastable shapes at loc(unknown)
	 [[{{node loss/conv2d_73_loss/mul}}]]
0 successful operations.
0 derived errors ignored.

In [ ]:
print("Train shapes:", X1_train.shape, X2_train.shape, y_train.shape)
print("Val shapes:", X1_val.shape, X2_val.shape, y_val.shape)

In [ ]:
y_train = y_train.reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = y_val.reshape(-1, IMG_SIZE, IMG_SIZE, 1)